**UNFINISHED**

Notebook for the engineering team to use to calculate x and y for the arm (x is part of the arm between servos at alpha and beta, y is between beta and the grabber).

max_range -> returns the maximum range the arm can extend to with the given values for x and y and constrained to 20 <= alpha <= 90 (degrees)
beta_clockwise_moments -> returns the net clockwise moment acting on the servo at beta
alpha_clockwise_moments -> returns the net clockwise moment acting on the servo at alpha

check_lengths -> checks the above three functions return answers that would mean the arm can attain sufficient range within the bounds of alpha and beta.

apologies for the horrbile alpha_clockwise_moments function, but i tried to keep the function formatted the same as the equation sent on discord

**Notes:**
Gradient ascent being used for finding the range has to check a lot of alpha values, so it there is an alpha value available in which it is not possible to touch the table then a complex number will be returned and we'll get an error. We want to be able to touch the table for all values of alpha in the range, so we know that the values for x and y are invalid if it crashes.

There is an issue with antclockwise moments which is going to be a little more tricky. Will investigate further later.

In [43]:
from math import cos, sin, atan, pi, radians  # all angles in radians
from itertools import product

In [44]:
def max_range(x, y, iterations=20, eta=0.1):  # eta is like learning rate
  get_range = lambda alpha, x, y : x * cos(alpha) + (y**2 - (0.1 - x*sin(alpha))**2)**0.5  # if we want to speed this up we can do it algerbraically, but this is more versatile imo
  get_range_grad = lambda alpha, x, y : ( (x * cos(alpha)) * (0.1 - x * sin(alpha)) )/( y**2 - (0.1 - x * sin(alpha))**2 )**0.5 - x * sin(alpha)

  max_alpha_val = pi/9
  for i in range(iterations):
    max_alpha_val += eta * get_range_grad(max_alpha_val, x, y)  # gradient ascent approximation
    max_alpha_val = max(min(max_alpha_val, pi/4), pi/9)  # constrain alpha to range

  return get_range(max_alpha_val, x, y)

In [45]:
SERVO_TORQUE = 1.2748

In [46]:
BETA_ANTICLOCKWISE_MOMENT = SERVO_TORQUE
beta_clockwise_moments = lambda alpha, beta, y : 11.772 * cos(alpha + beta - pi) * (0.0015 * y**2 + 0.385 * y + 2.75)  # this should be true for all alpha and beta in range

In [47]:
ALPHA_ANTICLOCKWISE_MOMENT = SERVO_TORQUE
alpha_clockwise_moments = lambda alpha, beta, x, y : 11.772 * ((x * 0.003) * (x / 2) * cos(alpha) + (0.055) * (x) \
                                                     * cos(alpha) + (y + 0.003) * ((x * sin(alpha) + y * sin(alpha + beta - pi) / 2) + (x * cos(alpha) \
                                                     + y * cos(alpha + beta - pi) / 2)**2)**0.5 * cos(atan((x * sin(alpha) + y * sin(alpha + beta -pi) / 2) \
                                                     / (x * cos(alpha) + y * cos(alpha + beta - pi) / 2))) + (0.11) * ((x * sin(alpha) + y * sin(alpha + beta - pi))**2 \
                                                     + (x * cos(alpha) + y * cos(alpha + beta - pi))**2)**0.5 * cos(atan((x * sin(alpha) + y * sin(alpha + beta - pi)) \
                                                     / (x * cos(alpha) + y * cos(alpha + beta - pi)))) + (0.275) * ((x * sin(alpha) + (y + 0.1) * sin(alpha + beta - pi))**2 \
                                                     + (x * cos(alpha) + (y + 0.1) * cos (alpha + beta - pi))**2)**0.5 * cos(atan((x * sin(alpha) + (y + 0.1) * sin(alpha \
                                                     + beta - pi)) / (x * cos(alpha) + (y + 0.1) * cos(alpha + beta - pi)))))  # could improve speed here by evaling cos(alpha, beta ... ) before

In [48]:
def check_lengths(x, y, iterations=20, eta=0.1):

  max_r = max_range(x, y, iterations=iterations, eta=eta)
  max_bcm = max([beta_clockwise_moments(radians(alpha), radians(beta), y) for alpha, beta in product(list(range(20,91)), list(range(10,161)))])
  max_acm = max([alpha_clockwise_moments(radians(alpha), radians(beta), x, y) for alpha, beta in product(list(range(20,91)), list(range(10,161)))])

  passed = True

  if 0.5 <= max_r:
    print(f"Range {max_r} passed")
  else:
    print(f"Range {max_r} did not pass")
    passed = False
  if BETA_ANTICLOCKWISE_MOMENT >= max_bcm:
    print(f"Max beta torque {max_bcm} passed")
  else:
    print(f"Max beta torque {max_bcm} did not pass")
    passed = False
  if ALPHA_ANTICLOCKWISE_MOMENT >= max_acm:
    print(f"Max alpha torque {max_acm} passed")
  else:
    print(f"Max alpha torque {max_acm} did not pass")
    passed = False
  
  return passed

In [49]:
x = 1
y = 1  # Insert vals to test here
print("Passed:", check_lengths(x, y))

Range 1.9099638458691954 passed
Max beta torque 36.922878 did not pass
Max alpha torque 27.590914900688787 did not pass
Passed: False
